In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import math 
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score, train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve
from sklearn.linear_model import LogisticRegression, RidgeClassifier, Lasso
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from catboost import CatBoostClassifier
from scipy.stats import uniform as sp_randFloat
from scipy.stats import randint as sp_randInt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nba-datasets/combined_stats_19to20_with_target.csv
/kaggle/input/nba-datasets/combined_stats_20to21_with_target.csv
/kaggle/input/nba-datasets/combined_stats_21to22_with_target.csv
/kaggle/input/nbadatasets/test_data_ratios.csv
/kaggle/input/nbadatasets/train_data_ratios.csv


In [2]:
year1 = pd.read_csv('../input/nba-datasets/combined_stats_19to20_with_target.csv')
year2 = pd.read_csv('../input/nba-datasets/combined_stats_20to21_with_target.csv')
year3 = pd.read_csv('../input/nba-datasets/combined_stats_21to22_with_target.csv')

In [3]:
# train_data = pd.concat( map(pd.read_csv, ['../input/nba-datasets/combined_stats_19to20_with_target.csv', '../input/nba-datasets/combined_stats_20to21_with_target.csv']), ignore_index=True)
# print(f'The shape of the train_data is: {train_data.shape}\n')
train_data = pd.read_csv('../input/nbadatasets/train_data_ratios.csv')
train_data.head()

,G_OFF,MP_OFF,FG%_OFF,3P%_OFF,2P%_OFF,FT%_OFF,ORB_OFF,DRB_OFF,TRB_OFF,AST_OFF,...,DRB_DEF,TRB_DEF,AST_DEF,STL_DEF,BLK_DEF,TOV_DEF,PF_DEF,PTS_DEF,WON_MORE_THAN_HALF,Eastern Conference
0,67,243.0,0.449,0.333,0.525,0.790,9.9,33.4,43.3,24.0,...,35.9,47.0,25.0,9.0,6.4,15.0,21.0,119.7,0,1
1,72,242.1,0.461,0.364,0.522,0.801,10.7,35.4,46.1,23.0,...,34.0,44.3,22.4,7.1,5.5,15.2,20.7,107.3,1,1
2,72,242.8,0.448,0.343,0.524,0.745,10.6,37.3,47.9,24.5,...,35.3,45.9,22.6,7.7,5.3,12.8,21.1,112.3,0,1
3,65,242.3,0.434,0.352,0.489,0.748,11.0,31.8,42.8,23.8,...,35.0,46.0,26.6,8.0,5.0,14.4,20.6,109.6,0,1
4,65,241.2,0.447,0.348,0.511,0.755,10.5,31.4,41.9,23.2,...,35.4,45.5,25.4,8.2,5.9,18.3,19.2,109.9,0,1


In [4]:
test_data = pd.read_csv('../input/nbadatasets/test_data_ratios.csv')
answers = test_data['WON_MORE_THAN_HALF']
# test_data = year3
test_data.drop('WON_MORE_THAN_HALF', axis = 1, inplace = True)\
# test_data.drop('WON_MORE_THAN_HALF')
test_data.head()

,G_OFF,MP_OFF,FG%_OFF,3P%_OFF,2P%_OFF,FT%_OFF,ORB_OFF,DRB_OFF,TRB_OFF,AST_OFF,...,ORB_DEF,DRB_DEF,TRB_DEF,AST_DEF,STL_DEF,BLK_DEF,TOV_DEF,PF_DEF,PTS_DEF,Eastern Conference
0,82,240.3,0.470,0.374,0.531,0.812,10.0,33.9,44.0,24.6,...,10.2,33.6,43.9,25.4,7.0,4.5,12.8,20.3,112.4,0
1,82,242.7,0.466,0.356,0.547,0.816,10.5,35.5,46.1,24.8,...,10.5,33.3,43.7,21.6,7.5,4.6,13.8,19.4,104.5,1
2,82,240.9,0.475,0.361,0.538,0.805,10.3,34.1,44.4,25.3,...,11.3,32.8,44.1,23.8,7.9,4.9,13.3,19.7,112.1,1
3,82,242.4,0.468,0.365,0.542,0.740,10.8,33.7,44.6,28.1,...,11.4,35.6,47.0,26.5,7.5,4.6,15.0,19.6,114.9,1
4,82,240.6,0.480,0.369,0.535,0.813,8.7,33.7,42.3,23.9,...,9.3,33.9,43.2,25.0,7.1,5.0,13.1,18.2,112.0,0


In [5]:
# teams = test_data['Team']
# answers = test_data['WON_MORE_THAN_HALF']

# Now drop the 'Id' column since it's unnecessary for  the prediction process.
# train_data.drop("Team", axis = 1, inplace = True)
# test_data.drop("Team", axis = 1, inplace = True)


In [6]:
# X = train_data.iloc[:, :30]
# y = train_data.WON_MORE_THAN_HALF
# x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
train_data = pd.read_csv('../input/nbadatasets/train_data_ratios.csv')
test_data = pd.read_csv('../input/nbadatasets/test_data_ratios.csv')
X_train = train_data.drop(['WON_MORE_THAN_HALF'], axis=1)
y_train = train_data['WON_MORE_THAN_HALF']
X_test = test_data.drop(['WON_MORE_THAN_HALF'], axis=1)
y_test = test_data['WON_MORE_THAN_HALF']

In [7]:
# x_train.shape, y_train.shape

In [8]:
# x_test.shape, y_test.shape

**Base Cat Boost Model**

In [9]:
cb_base_model = CatBoostClassifier(verbose=False)
cb_base_model.fit(X_train, y_train.values.ravel())
preds = cb_base_model.predict(X_test)
acc = accuracy_score(y_test, preds)
auc = roc_auc_score(y_test, preds)
print(f'The accuracy score using a test_size of 20% is {acc}')
print(f'The area under the curve score using a test size of 20% is {auc}')
y_preds = cb_base_model.predict(test_data)

The accuracy score using a test_size of 20% is 0.8333333333333334
The area under the curve score using a test size of 20% is 0.8472222222222222


In [10]:
y_preds

array([0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0,
       1, 1, 0, 0, 0, 1, 1, 0])

In [11]:
accuracy_score(answers, y_preds)

0.8333333333333334

**Hyperparameter Tuning of Cat Boost Classifer**

In [12]:
cb_tuned_model = CatBoostClassifier(verbose=False,max_depth=6, n_estimators=1000, learning_rate=0.01, l2_leaf_reg=2)
cb_tuned_model.fit(X_train, y_train.values.ravel())
preds = cb_tuned_model.predict(X_test)
acc = accuracy_score(y_test, preds)
auc = roc_auc_score(y_test, preds)
print(f'The accuracy score is {acc}')
print(f'The area under the curve score is {auc}')
y_preds = cb_tuned_model.predict(test_data)

The accuracy score is 0.7666666666666667
The area under the curve score is 0.7916666666666666


In [13]:
accuracy_score(answers, y_preds)

0.7666666666666667

In [14]:
lr_base_model = LogisticRegression()
lr_base_model.fit(X_train, y_train)
preds = lr_base_model.predict(X_test)
acc = accuracy_score(y_test, preds)
auc = roc_auc_score(y_test, preds)
print(f'The accuracy score is {acc}')
print(f'The area under the curve score is {auc}')
# y_preds = lr_base_model.predict(test_data)

The accuracy score is 0.9333333333333333
The area under the curve score is 0.9305555555555555


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [15]:
accuracy_score(answers, y_preds)

0.7666666666666667